An example yaml config

```yaml
model:
  model_name: batch_reactor_test
  model_type: continuous
  constants:
    mu_m: 0.02
    K_m: 0.05
    K_i: 5.0
    v_par: 0.004
    Y_p: 1.2

  state_variables:
    X_s:
      init_val: 1.0
      init_val_lower: 0.9
      init_val_upper: 1.1
      rhs: mu_S*X_s - inp/V_s*X_s
      shape: (1,) # default value, can be omitted
      scaling: 1 # default value, can be omitted
    S_s:
      init_val: 0.5
      init_val_lower: 0.4
      init_val_upper: 0.6
      rhs: mu_S*X_s/Y_x - v_par*X_s/Y_p + inp/V_s*(S_in-S_s)
    P_s:
      init_val: 0.0
      init_val_lower: 0.0
      init_val_upper: 0.2
      rhs: v_par*X_s - inp/V_s*P_s
    V_s:
      init_val: 120.0
      init_val_lower: 110.0
      init_val_upper: 130.0
      rhs: inp

  control_variables:
    inp:
      shape: (1,) # default value, can be omitted

  user_defined_parameters:
    - Y_x
    - S_in
  aux_variables:
    mu_S:
      expr: mu_m*S_s/(K_m+S_s+(S_s**2/K_i))
```
---- 

# Optional: Loading config from a file

In [1]:
import yaml
wd = None
with open("model_configs/model_config_cstr.yaml") as f:
    wd = yaml.load(f, Loader=yaml.FullLoader)
wd


{'model': {'model_name': 'cstr_test',
  'model_type': 'continuous',
  'constants': {'K0_ab': 1287000000000.0,
   'K0_bc': 1287000000000.0,
   'K0_ad': 9043000000.0,
   'R_gas': 0.0083144621,
   'E_A_ab': 9758.3,
   'E_A_bc': 9758.3,
   'E_A_ad': 8560.0,
   'H_R_ab': 4.2,
   'H_R_bc': -11.0,
   'H_R_ad': -41.85,
   'Rou': 0.9342,
   'Cp': 3.01,
   'Cp_k': 2.0,
   'A_R': 0.215,
   'V_R': 10.01,
   'm_k': 5.0,
   'T_in': 130.0,
   'K_w': 4032.0,
   'C_A0': 5.1},
  'state_variables': {'C_a': {'init_val': [0.8],
    'init_val_lower': [0.1],
    'init_val_upper': [2.0],
    'rhs': 'F*(C_A0 - C_a) -K_1*C_a - K_3*(C_a**2)',
    'shape': '(1,1)',
    'scaling': 1},
   'C_b': {'init_val': [0.5],
    'init_val_lower': [0.1],
    'init_val_upper': [2.0],
    'shape': '(1,1)',
    'rhs': '-F*C_b + K_1*C_a - K_2*C_b'},
   'T_R': {'init_val': [134.14],
    'init_val_lower': [50.0],
    'init_val_upper': [150.0],
    'shape': '(1,1)',
    'rhs': '((K_1*C_a*H_R_ab + K_2*C_b*H_R_bc + K_3*(C_a**2)*H_R_ad

# Define your model
- Step 1: Set meta-data for the model including name, type, number of variables of each type
- Step 2: Define in details each variable


In [2]:
from vis_utility import *
var_grid, widget_dict = var_num_grid_generator(default_wd = wd)
display(var_grid)


GridspecLayout(children=(Button(description='Model Specification', layout=Layout(grid_area='widget001', width=…

## Define constants if any including name and value

In [3]:
# define details of each variable/constant/parameter
num_constants = widget_dict["num_constants"].value
constant_grid, _widget_dict = constants_grid_generator(num_constants, default_wd = wd)
if num_constants > 0:
    display(constant_grid)
    widget_dict.update(_widget_dict)

GridspecLayout(children=(Button(description='Constants Definition', layout=Layout(grid_area='widget001', width…

## Define each state variable if any including name, initial value, lower and upper bounds, RHS (right-hand-side equation), shape, scaling (used in MPC), whether the bound is soft or not

In [4]:
num_state_variables = widget_dict["num_state_variables"].value
sv_grid, _widget_dict = state_variables_grid_generator(num_state_variables, default_wd=wd)
if num_state_variables > 0:
    display(sv_grid)
    widget_dict.update(_widget_dict)
    
    

GridspecLayout(children=(Button(description='State Variables Definition', layout=Layout(grid_area='widget001',…

## Define each control variable if any including name and shape

In [5]:

num_control_variables = widget_dict["num_control_variables"].value
if num_control_variables > 0:
    control_grid, _widget_dict = control_variables_grid_generator(num_control_variables, default_wd=wd)
    display(control_grid)
    widget_dict.update(_widget_dict)



2


GridspecLayout(children=(Button(description='Control Variables Definition', layout=Layout(grid_area='widget001…

# Define user defined parameters if any; value of each parameter is a list from which we can sample one to initialize

In [6]:
num_parameters = widget_dict["num_parameters"].value
if num_parameters > 0:
    parameter_grid, _widget_dict = parameters_grid_generator(num_parameters, default_wd=wd)
    display(parameter_grid)
    widget_dict.update(_widget_dict)


GridspecLayout(children=(Button(description='User Defined Parameter Definition', layout=Layout(grid_area='widg…

# Define auxiliary variables if any including name and its concrete definition based on other variables

In [7]:
num_aux_variables = widget_dict["num_aux_variables"].value
if num_aux_variables > 0:
    aux_variable_grid, _widget_dict = aux_variable_grid_generator(num_aux_variables, default_wd=wd)
    display(aux_variable_grid)
    widget_dict.update(_widget_dict)

GridspecLayout(children=(Button(description='Auxiliary Variables Definition', layout=Layout(grid_area='widget0…

# An example yaml config for simulator

```yaml

simulator:
  parameters:
    integration_tool: 'cvodes'
    abstol: 1.0e-10
    reltol: 1.0e-10
    t_step: 1.0
```
---- 

# Define the simulator
- Step 1: specify number of parameters in order to instantiate a simulate
- Step 2: define each parameter

In [8]:
num_simulator_parameters = BoundedIntText(value=(1 if wd is None else len(list(wd["simulator"].get("parameters", {}).keys()))),
                                           min=0,
                                           step=1,
                                           description='Num. Simulator Parameters:',
                                           style={'description_width': 'initial'},
                                           disabled=False)
display(num_simulator_parameters)

BoundedIntText(value=4, description='Num. Simulator Parameters:', style=DescriptionStyle(description_width='in…

## Define parameters to initilize a simulator including name, value, and whether it is numeric

In [9]:
widget_dict.update({"num_simulator_parameters": num_simulator_parameters.value})

if num_simulator_parameters.value > 0:
    sim_para_grid, _widget_dict = simulator_parameter_grid_generator(num_simulator_parameters.value, default_wd=wd)
    display(sim_para_grid)
    widget_dict.update(_widget_dict)         

GridspecLayout(children=(Button(description='Simulator Parameters', layout=Layout(grid_area='widget001', width…

# An example yaml config for reward function

```yaml
reward:
  step_reward:
    P_s:
      expr: P_s
      coef: 1.0
  terminal_reward:
    P_s:
      expr: P_s
      coef: 1.0
  input_reward:
    inp: 1.0
```
---- 

## Reward function may contain three components: 
- State step reward - define over state variables or expressions and will be calculated cumulatively during the execution
- State terminal reward - similar as step reward except it only calculated in the terminal states
- Action/input reward - to measure policy smoothness by calculating L1 distance between two actions/inputs in sequence

In [10]:
sv_names = [widget_dict[f"sv_{i}_def"].value for i in range(widget_dict["num_state_variables"].value)]
in_names = [widget_dict[f"cv_{i}_def"].value for i in range(widget_dict["num_control_variables"].value)]

state_rewards_grid, _widget_dict = state_rewards_grid_generator(sv_names, default_wd=wd)
if len(sv_names) > 0:
    widget_dict.update(_widget_dict)
    display(state_rewards_grid)
            

GridspecLayout(children=(Button(description='Define State Reward', layout=Layout(grid_area='widget001', width=…

In [11]:
input_rewards_grid, _widget_dict = input_rewards_grid_generator(in_names, default_wd=wd)
if len(in_names) > 0:
    widget_dict.update(_widget_dict)
    display(input_rewards_grid)

GridspecLayout(children=(Button(description='Define Input Reward', layout=Layout(grid_area='widget001', width=…

# An example yaml config for MPC

```yaml
mpc:
  setup:
    n_horizon: 20
    n_robust: 0
    open_loop: 0
    t_step: 1.0
    state_discretization: collocation
    collocation_type: radau
    collocation_deg: 2
    collocation_ni: 2
    store_full_solution: True
  bounds:
    X_s:
      lower: 0.0 # required
      upper: 3.7 # required
      soft: False # default, can be omitted
    S_s:
      lower: -0.01
      upper: 2.0
    P_s:
      lower: 0.0
      upper: 3.0
    V_s:
      lower: 0.0
      upper: 150.0
    inp:
      lower: 0.0
      upper: 0.2
  uncertainities:
    Y_x: [0.5, 0.4, 0.3]
    S_in: [200.0, 220.0, 180.0]

```
---- 

# Define a MPC controller
- Step 1: Specify number of parameters in order to instantiate a MPC controller
- Step 2: Assign values to these parameters

In [12]:
num_MPC_parameters = BoundedIntText(value=(1 if wd is None else len(list(wd["mpc"].get("setup", {}).keys()))),
                                    min=0,
                                    step=1,
                                    description='Num. MPC Parameters:',
                                    style={'description_width': 'initial'},
                                    disabled=False)
display(num_MPC_parameters)



BoundedIntText(value=9, description='Num. MPC Parameters:', style=DescriptionStyle(description_width='initial'…

## Define parameters to initilize a MPC controller including name, value, and whether it is numeric

In [13]:
widget_dict.update({"num_MPC_parameters": num_MPC_parameters.value})

if num_MPC_parameters.value > 0:
    MPC_para_grid, _widget_dict = MPC_parameter_grid_generator(num_MPC_parameters.value, default_wd=wd)
    display(MPC_para_grid)
    widget_dict.update(_widget_dict) 

GridspecLayout(children=(Button(description='MPC Parameters', layout=Layout(grid_area='widget001', width='auto…

In [14]:
MPC_config_grid, _widget_dict = MPC_config_grid_generator(sv_names, in_names, default_wd=wd)
display(MPC_config_grid)
widget_dict.update(_widget_dict)

GridspecLayout(children=(Button(description='MPC Config', layout=Layout(grid_area='widget001', width='auto'), …

# An example yaml config for estimator

```yaml
estimator:
  type: StateFeedback # StateFeedback, EKF(Extended Kalman Filter), ..
  parameters: null
```
---- 
- Step 1: Specify number of parameters to initialize an estimator
- Step 2: Define each parameter including name, value, and type

In [15]:
value = 1
if wd is not None:
    est_parameters = wd["estimator"].get("parameters", {})
    if est_parameters is None: value = 0
    else: value = len(est_parameters.keys())

num_estimator_parameters = BoundedIntText(value=value,
                                          min=0,
                                          step=1,
                                          description='Num. Estimator Parameters:',
                                          style={'description_width': 'initial'},
                                          disabled=False)
display(num_estimator_parameters)

BoundedIntText(value=0, description='Num. Estimator Parameters:', style=DescriptionStyle(description_width='in…

In [16]:
widget_dict.update({"num_estimator_parameters": num_estimator_parameters.value})
estimator_para_grid, _widget_dict = estimator_parameter_grid_generator(num_estimator_parameters.value, default_wd=wd)
display(estimator_para_grid)
widget_dict.update(_widget_dict)

GridspecLayout(children=(Button(description='Estimator Parameters', layout=Layout(grid_area='widget001', width…

# Save to a yaml file once we define everything


In [17]:
save2yaml(widget_dict, "tmp.yaml")